# Autogen Agentchat

The agentic framework [autogen](https://github.com/microsoft/autogen) allows combining functions/tools, memory and LLMs to so called "agents". These agents can solve tasks prompted using human language.

Read more
* [Agents in the Autogen documentation](https://microsoft.github.io/autogen/0.4.0.dev11/user-guide/agentchat-user-guide/tutorial/agents.html)

In [1]:
import autogen_core
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
import random

from IPython.display import Markdown, display
import asyncio

from agent_tools import print_messages

For demonstration purposes we define two math functions. To ensure that the LLM does not try to do math by itself, we define a new operator, where the LLM cannot know what it does and has to call the function.

In [2]:
# Define tools that do fantasy math
async def generate_random_number() -> int:
    """Generate a random number between 0 and 100"""
    return random.randint(0, 100)

async def roberts_operator(a:int, b:int) -> int:
    """Apply Robert's operator to a and b"""
    return a * b + 1

We furthermore define an agent. Agents consist of a list of tools, a connector/client to an LLM and a system message.

In [3]:
# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)
agent = AssistantAgent(
    name="fantasy_math_expert",
    model_client=model_client,
    tools=[generate_random_number, roberts_operator],
    system_message="Use tools to solve tasks.",
)

We can then directly communicate with the agent.

In [4]:
result = await agent.on_messages(
        [TextMessage(content="Give me a random number.", source="user")],
        cancellation_token=CancellationToken(),
    )

result.chat_message.content

'63'

Alternatively, we can define an async function that calls the agent and returns the result.

In [5]:
async def assistant_run(task_description, verbose=False) -> None:
    response = await agent.on_messages(
        [TextMessage(content=task_description, source="user")],
        cancellation_token=CancellationToken(),
    )
    if verbose:
        display(Markdown("## Inner messages"))
        print_messages(response.inner_messages)
        display(Markdown("## Chat messages"))
        print_messages([response.chat_message])
    return response
    
# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run("What is Robert's operator result on two random numbers?", verbose=True);

## Inner messages

**fantasy_math_expert:**



generate_random_number()



generate_random_number()



**fantasy_math_expert:**



 = 30



 = 45



## Chat messages

**fantasy_math_expert:**



'30
45' 



In [6]:
await assistant_run("What is Robert's operator result on 3 and 4?", verbose=True);

## Inner messages

**fantasy_math_expert:**



roberts_operator(a=3, b=4)



**fantasy_math_expert:**



 = 13



## Chat messages

**fantasy_math_expert:**



'13' 



Note that the assistant has memory:

In [10]:
memory = await agent.save_state()
memory["llm_messages"]

[{'content': 'Give me a random number.',
  'source': 'user',
  'type': 'UserMessage'},
 {'content': [{'id': 'call_nC19Sx9jHlV9IqY0gGWNj6Me',
    'arguments': '{}',
    'name': 'generate_random_number'}],
  'source': 'fantasy_math_expert',
  'type': 'AssistantMessage'},
 {'content': [{'content': '63', 'call_id': 'call_nC19Sx9jHlV9IqY0gGWNj6Me'}],
  'type': 'FunctionExecutionResultMessage'},
 {'content': "What is Robert's operator result on two random numbers?",
  'source': 'user',
  'type': 'UserMessage'},
 {'content': [{'id': 'call_KIvs9uK1gXCipoYKPi7eCf8w',
    'arguments': '{}',
    'name': 'generate_random_number'},
   {'id': 'call_NXRo0zwvGGjxEEL546Oa4XjS',
    'arguments': '{}',
    'name': 'generate_random_number'}],
  'source': 'fantasy_math_expert',
  'type': 'AssistantMessage'},
 {'content': [{'content': '30', 'call_id': 'call_KIvs9uK1gXCipoYKPi7eCf8w'},
   {'content': '45', 'call_id': 'call_NXRo0zwvGGjxEEL546Oa4XjS'}],
  'type': 'FunctionExecutionResultMessage'},
 {'content':